In [32]:
import pandas as pd
import csv
import os
import pymysql
from pathlib import Path
from openpyxl import Workbook
import pandas as pd
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from database_read import read
from sqlalchemy import create_engine
import re
from datetime import datetime
# origin = pd.read_excel('E:/after_sale/总售后2022-04-30.xlsx')
origin = read('after_sales','new_after_salesv2_2021')

origin['订单号'] = origin['订单号'].astype(str)
origin.head()

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片
0,1,2021-01-01,2022-05-11,王帆,amazon优瑞斯特,111-0252067-5192266,USAN1011322-2,1,已解决,5-3-Y$U99&1-3-K,3-1-0$USAN1011322-5num1,0.00,买家说没收到，最后发现送错门洞，箱子被雨淋湿且包装破损，要求补寄新件,已上传
1,2,2021-01-01,2022-05-10,章珊珊,amazon优瑞斯特,111-1636759-4867407,HOG102201-04,1,已解决,1-4-K,3-1-0$USAN1016483-6num1,0.00,None,未上传
2,3,2021-01-01,2022-05-13,曹玲玲,amazon赫曼,111-1930735-3781001,USAN1017103-3,1,已解决,5-3-Y$A#B,1-1-0&3-1-0$USAN1017103-3num1,135.00,外包装和里面产品都破损，无法使用，补寄了之后买家又不要，想要退全款,未检查
3,4,2021-01-01,2022-05-13,曹玲玲,amazon赫曼,111-1930735-3781001,USAN1017103-4,2,已解决,4-3-M,3-1-0$USAN1017103-4num1&1-1-0,135.00,外包装和里面产品都破损，无法使用，补寄了之后买家又不要，想要退全款,未检查
4,5,2021-11-15,2022-05-16,金嘉豪,amazon信盒,111-2118495-4433024,USAN1029931-3,1,已解决,4-4-K,1-2-0&3-1-0$USAN1029931-3num1&2-1-0ship1,207.88,第一次是未出库截回，于是补寄新件，pending后买家不要了，最后包裹还是到了,未上传


In [33]:
origin.dtypes

id          int64
登记日期       object
更新日期       object
登记人        object
店铺         object
订单号        object
SKU        object
序号         object
订单状态       object
顾客反馈       object
客服操作       object
Refund    float64
备注         object
是否上传图片     object
dtype: object

In [34]:
origin[origin['订单号'] == 'NF10834863303']

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片


In [35]:
origin_buji = origin[origin["客服操作"].str.contains('3-5-0')]
origin_buji = origin_buji.reset_index(drop = True)
origin_buji['补寄配件次数'] = 0
# origin_buji['补寄SKU'] = ' '
origin_buji

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片,补寄配件次数
0,82,2021-01-01,2021-01-01,孙紫馨,amazon优瑞斯特,114-8285500-6937869,USAN1011299-3,1,已解决,,3-5-0,0.00,None,未检查,0
1,92,2021-01-02,2021-01-02,敖睿洁,amazon优瑞斯特,111-4501807-2218608,USAN1015826-2,1,已解决,,3-5-0,0.00,None,未检查,0
2,170,2021-01-03,2021-01-03,敖睿洁,amazon优瑞斯特,112-8590071-3421018,USAN1017139-6,1,已解决,,3-5-0,0.00,None,未检查,0
3,213,2021-01-04,2022-05-09,王俊杰,amazon信盒,111-9302543-0591421,HOG101439-02,1,已解决,&6-7-S$L99,3-5-0$L99ship1,0.00,少螺丝包,未检查,0
4,221,2021-01-04,2022-05-12,郄思颖,amazon宫本,112-4670525-8085040,USAN1015826-1,1,已解决,6-7-S$A2#J#S99,1-2-0&3-5-0$A2#J#S99ship1&3-2-0$USAN1015826-1n...,179.99,0,未检查,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,56611,2021-12-31,2021-12-31,徐志炜,amazon信盒,114-9269699-6625808,HOG100555-01,1,已解决,,3-5-0,0.00,凳腿*1，螺丝吧,未检查,0
3102,56616,2021-12-31,2022-05-11,鲁佳,walmart赫曼,3863473557103,USAN1022780-8,1,已解决,7-4-Z$W,3-5-0$Wship1,0.00,遥控器型号不对,未上传,0
3103,56628,2021-12-31,2021-12-31,李娜,walmart优瑞斯特,5863559215726,USAN1022706-6,1,已解决,,3-5-0,0.00,螺丝包,未检查,0
3104,56632,2021-12-31,2021-12-31,李娜,walmart优瑞斯特,9863380139721,USAN1031424-5,1,已解决,,3-5-0,0.00,螺丝包,未检查,0


In [36]:
origin_buji[origin_buji['订单号'] == 'NF10834863303']

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片,补寄配件次数


In [37]:
def find_all(string, sub):
    start = 0
    pos = []
    while True:
        start = string.find(sub, start)
        if start == -1:
            return pos
        pos.append(start)
        start += len(sub)

In [38]:
inventory_in_buji = pd.DataFrame(columns=['id', '登记日期', '更新日期', '登记人', '店铺', '订单号', 'SKU', '序号', '订单状态', '顾客反馈', '客服操作','操作代码','Refund', '备注'])
dengji_buji = pd.DataFrame(columns=['id', '登记日期', '更新日期', '登记人', '店铺', '订单号', 'SKU', '序号', '订单状态', '顾客反馈', '客服操作', 'Refund', '备注','补寄sku数量','补寄配件次数','数据库补寄数量'])
code = ['3-5-0']
origin_buji.loc[:,'补寄配件次数'] = 0
import time
for i in range(len(origin_buji)):
    code_split_list = origin_buji.loc[i,'客服操作'].split('&')
#     code_split_list = '3-1-0$ESAN1024542num1&2-4-0'.split('&')
    for j in code :
        num_sum = 0
        for k in range(len(code_split_list)):
#             split_num = code_split_list[k].split('num')
#             if len(split_num)>1:
#                 origin_buji.loc[i,'补寄SKU'] ='%s,%s' %(origin_buji.loc[i,'补寄SKU'])
            if j in  code_split_list[k]:

                if "num" not in code_split_list[k] and "ship" not in code_split_list[k]:

                     num_sum  = num_sum+1
                if len(find_all(code_split_list[k],'num')) == 1:
                    num = code_split_list[k].index('num')
                    num_n1 = int(code_split_list[k][num+3:num+4])
                    num_sum =  num_n1
#                     origin_buji.loc[i,'补寄数量'] = num_n1
                if 'ship' in code_split_list[k]  : 
                    num_ship = code_split_list[k].index('ship')
                    ship_n = int(code_split_list[k][num_ship+4:num_ship+5])
                    num_sum = num_sum + ship_n
                
#                     origin_buji.loc[i,'补寄数量'] = ship_n
        origin_buji.loc[i,'补寄配件次数'] = origin_buji.loc[i,'补寄配件次数']+num_sum
#     num_sum1 = 0
#     for k in range(len(code_split_list)):
#         if '3-5-0' in  code_split_list[k]:
#             if "num" not in code_split_list[k] and "ship" not in code_split_list[k]:
#                  num_sum1  = num_sum1+1
#             if len(find_all(code_split_list[k],'num')) == 1:
#                 num1 = code_split_list[k].index('num')
#                 num_n2 = int(code_split_list[k][num1+3:num1+4])
#                 num_sum1 =  num_n2
# #                     origin_buji.loc[i,'补寄数量'] = num_n1
#             if 'ship' in code_split_list[k]  : 
#                 num_ship1 = code_split_list[k].index('ship')
#                 ship_n1 = int(code_split_list[k][num_ship1+4:num_ship1+5])
#                 num_sum1 = num_sum1 + ship_n1
#                     origin_buji.loc[i,'补寄数量'] = ship_n
#     origin_buji.loc[i,'补寄配件次数'] = origin_buji.loc[i,'补寄配件次数']+num_sum1
     
#     print(origin_buji.loc[i,'操作代码'],origin_buji.loc[i,'补寄sku数量'])

now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time())) 
origin_buji.to_excel(excel_writer="D:/work/origin_buji/国内"+now+r".xlsx", index=False,encoding='utf-8')

In [39]:
# origin_buji['补寄数量'] = origin_buji['补寄sku数量']+origin_buji['补寄配件次数']
origin_buji[origin_buji['订单号'] == 'NF10834863303']

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片,补寄配件次数


In [40]:
origin_buji_table = origin_buji.pivot_table(index = '订单号',aggfunc = { '补寄配件次数':'sum'} )
origin_buji_table = origin_buji_table.reset_index()


In [41]:
origin_buji_table[origin_buji_table['订单号'] == '7863636525433']

,订单号,补寄配件次数


In [42]:
inventory_buji= pd.read_excel('D:/work/国内补寄/总配件.xlsx')

inventory_buji['订单号'] = inventory_buji['订单号'].astype(str)
inventory_buji.dtypes

ID              int64
日期     datetime64[ns]
订单号            object
SKU            object
详情             object
寄件人            object
店铺             object
备注            float64
dtype: object

In [43]:
for i in range(len(inventory_3)):
    if str(inventory_3.iloc[i,2]) not in origin_buji_big['订单号'].tolist():
        print(i)

615
1203
1276
1365
1383
1401
1417
1420
1428


In [44]:
inventory_buji[inventory_buji['订单号'] == 'NF10834863303']

,ID,日期,订单号,SKU,详情,寄件人,店铺,备注
1110,1111,2022-04-06,NF10834863303,USAN1022653-1,partR*6,孟坤,Shopify-nextfur,NaN


In [45]:
inventory_buji[inventory_buji['SKU'] == 'USAN1031424-3']

,ID,日期,订单号,SKU,详情,寄件人,店铺,备注
174,175,2022-03-16,111-9363596-7629835,USAN1031424-3,床腿H*1,邵陈琪,赫曼,NaN
218,219,2022-02-09,112-1284598-6284252,USAN1031424-3,"instructions,feetforfootboard,middlebrackets,s...",张科俊,森月,NaN
219,220,2022-02-21,112-1284598-6284252,USAN1031424-3,金属脚,张科俊,信盒,NaN
331,332,2022-03-24,112-8067548-7271455,USAN1031424-3,PartKx2,张科俊,信盒,NaN
645,646,2022-02-09,114-4855233-8678612,USAN1031424-3,五金件,李婷婷,信盒,NaN
759,760,2022-01-29,2863468564809,USAN1031424-3,床腿F1、床腿I和床尾连接扣K、铁片J、螺丝包1个,鲁佳,Walmart-赫曼,NaN
765,766,2022-02-12,2870149219505,USAN1031424-3,螺丝包,李永芳,Walmart-优瑞斯特,NaN
766,767,2022-02-16,2870149219505,USAN1031424-3,"partH*2，partI和partL,partF1，J*6，K*2",李永芳,Walmart-优瑞斯特,NaN
784,785,2022-03-28,3870189564353,USAN1031424-3,床腿partH*1,李永芳,Walmart-优瑞斯特,NaN
1256,1260,2022-04-25,111-8750933-4982636,USAN1031424-3,PartJx6,张科俊,赫曼,NaN


In [46]:
inventory_pivot = pd.pivot_table(inventory_buji, values=['SKU'], index=['订单号'], aggfunc='count')
inventory_pivot = inventory_pivot.reset_index()

inventory_pivot

,订单号,SKU
0,06-08329-92564,1
1,06-08389-60256,1
2,08-08324-81027,1
3,08-08324-81028,1
4,10-08131-23810,1
...,...,...
1420,NF10834863185,1
1421,NF10834863303,1
1422,NF10834863400,1
1423,NF10834863680,1


In [47]:
inventory_pivot[inventory_pivot['订单号'] == '7863636525433']

,订单号,SKU
1168,7863636525433,1


In [48]:
# dengji_buji = pd.DataFrame(columns=['id', '登记日期', '更新日期', '登记人', '店铺', '订单号', 'SKU', '序号', '订单状态', '顾客反馈', '客服操作','操作代码', 'Refund', '备注','补寄sku数量','补寄配件次数','补寄数量','数据库补寄数量'])
# inventory_in_buji = pd.DataFrame(columns=['id', '登记日期', '更新日期', '登记人', '店铺', '订单号', 'SKU', '序号', '订单状态', '顾客反馈', '客服操作','操作代码', 'Refund', '备注','补寄sku数量','补寄配件次数','补寄数量','数据库补寄数量'])

# import time
# for i in range(len(origin_buji)):
#     print(i)
#     if (origin_buji.iloc[i,5] in inventory_buji['订单号'].tolist()):
#         sku_sit = inventory_pivot['订单号'].tolist().index(origin_buji.iloc[i,5])
#         table_sit = origin_buji_table['订单号'].tolist().index(origin_buji.iloc[i,5])
#         if int(origin_buji_table.iloc[table_sit,1]) != int(inventory_pivot.iloc[sku_sit,1]) :
#             print(origin_buji.loc[i,'订单号'],'***********')
#             origin_output1 = pd.DataFrame(origin[origin['订单号']==origin_buji.iloc[i,5]])
#             origin_output1['补寄数量'] = origin_buji_table.iloc[table_sit,1]
#             origin_output1['数据库补寄数量'] = inventory_pivot.iloc[sku_sit,1]        
#             dengji_buji=dengji_buji.append(origin_output1)

#     else:
#         print(origin_buji.iloc[i,5],'-------')
#         table_sit = origin_buji_table['订单号'].tolist().index(origin_buji.iloc[i,5])
#         origin_output = pd.DataFrame(origin[origin['订单号']==origin_buji.iloc[i,5]])
#         origin_output['补寄数量'] = origin_buji_table.iloc[table_sit,1]
            
#         inventory_in_buji=inventory_in_buji.append(origin_output)


# #加个去重！！！！！


In [49]:
# dengji_buji


In [50]:
# now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time())) 

# # dengji_buji = dengji_buji.drop_duplicates(keep = False)

# dengji_buji.to_excel(excel_writer="E:/国内补寄/国内补寄校对/"+now+r"登记_补寄数量不符.xlsx", index=False,encoding='utf-8')
# inventory_in_buji.to_excel(excel_writer="E:/国内补寄/国内补寄校对/"+now+r"错填或不在国内补寄清单中的订单.xlsx", index=False,encoding='utf-8')

In [51]:
# origin_buji_big = origin_buji[origin_buji['补寄配件次数'] > 0]
origin_buji_big = origin_buji
origin_buji_big
inventory_buji_big =inventory_buji
inventory_3 = inventory_buji_big
origin_buji_big

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片,补寄配件次数
0,82,2021-01-01,2021-01-01,孙紫馨,amazon优瑞斯特,114-8285500-6937869,USAN1011299-3,1,已解决,,3-5-0,0.00,None,未检查,1
1,92,2021-01-02,2021-01-02,敖睿洁,amazon优瑞斯特,111-4501807-2218608,USAN1015826-2,1,已解决,,3-5-0,0.00,None,未检查,1
2,170,2021-01-03,2021-01-03,敖睿洁,amazon优瑞斯特,112-8590071-3421018,USAN1017139-6,1,已解决,,3-5-0,0.00,None,未检查,1
3,213,2021-01-04,2022-05-09,王俊杰,amazon信盒,111-9302543-0591421,HOG101439-02,1,已解决,&6-7-S$L99,3-5-0$L99ship1,0.00,少螺丝包,未检查,1
4,221,2021-01-04,2022-05-12,郄思颖,amazon宫本,112-4670525-8085040,USAN1015826-1,1,已解决,6-7-S$A2#J#S99,1-2-0&3-5-0$A2#J#S99ship1&3-2-0$USAN1015826-1n...,179.99,0,未检查,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,56611,2021-12-31,2021-12-31,徐志炜,amazon信盒,114-9269699-6625808,HOG100555-01,1,已解决,,3-5-0,0.00,凳腿*1，螺丝吧,未检查,1
3102,56616,2021-12-31,2022-05-11,鲁佳,walmart赫曼,3863473557103,USAN1022780-8,1,已解决,7-4-Z$W,3-5-0$Wship1,0.00,遥控器型号不对,未上传,1
3103,56628,2021-12-31,2021-12-31,李娜,walmart优瑞斯特,5863559215726,USAN1022706-6,1,已解决,,3-5-0,0.00,螺丝包,未检查,1
3104,56632,2021-12-31,2021-12-31,李娜,walmart优瑞斯特,9863380139721,USAN1031424-5,1,已解决,,3-5-0,0.00,螺丝包,未检查,1


In [52]:
origin_buji_big[origin_buji_big['订单号'] == '7871199027879']

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片,补寄配件次数


In [53]:
loudeng = inventory_3[~inventory_3['订单号'].astype(str).isin(origin_buji_big['订单号'])]
loudeng

,ID,日期,订单号,SKU,详情,寄件人,店铺,备注
0,1,2022-01-27,112-6466044-4167449,USAN1032096-11,床头柱B2,胡程程,赫曼,NaN
1,2,2022-01-25,114-4681848-2996252,USAN1026259-3,partJ,黄平,信盒,NaN
2,3,2022-01-11,CS378515836,USAN1029122-2,连接扣O2，O1,王荣荣,Wayfair-信盒贸易,NaN
3,4,2022-03-07,06-08329-92564,USAN1025836-1,螺丝partE.F,曹玲玲,eBay-zhirun2021,NaN
4,5,2022-03-21,08-08324-81027,USAN1034654-2,遥控器,邓园园,eBay-yaqinsale11,NaN
...,...,...,...,...,...,...,...,...
1478,1485,2022-05-09,112-3112726-2556257,USAN1011085-4,"PartP（排骨条卡扣）*12,PartQ（排骨条卡扣）*24",金晓强,赫曼,NaN
1479,1487,2022-05-09,112-5375617-8366622,USAN1023811-5,床腿H，挂片K*2,李莹莹,信盒,NaN
1480,1488,2022-05-09,114-7904736-4051433,USAN1026261-2,O塑料固定片*28,严鹏程,宫本,NaN
1481,1489,2022-05-09,CS377684547,USAN1029204-1,轮子*6,屠乐天,Wayfair-信盒贸易,NaN


In [54]:
loudeng1 = pd.merge(loudeng,inventory_3,on = '订单号')
loudeng1

,ID_x,日期_x,订单号,SKU_x,详情_x,寄件人_x,店铺_x,备注_x,ID_y,日期_y,SKU_y,详情_y,寄件人_y,店铺_y,备注_y
0,1,2022-01-27,112-6466044-4167449,USAN1032096-11,床头柱B2,胡程程,赫曼,NaN,1,2022-01-27,USAN1032096-11,床头柱B2,胡程程,赫曼,NaN
1,2,2022-01-25,114-4681848-2996252,USAN1026259-3,partJ,黄平,信盒,NaN,2,2022-01-25,USAN1026259-3,partJ,黄平,信盒,NaN
2,3,2022-01-11,CS378515836,USAN1029122-2,连接扣O2，O1,王荣荣,Wayfair-信盒贸易,NaN,3,2022-01-11,USAN1029122-2,连接扣O2，O1,王荣荣,Wayfair-信盒贸易,NaN
3,4,2022-03-07,06-08329-92564,USAN1025836-1,螺丝partE.F,曹玲玲,eBay-zhirun2021,NaN,4,2022-03-07,USAN1025836-1,螺丝partE.F,曹玲玲,eBay-zhirun2021,NaN
4,5,2022-03-21,08-08324-81027,USAN1034654-2,遥控器,邓园园,eBay-yaqinsale11,NaN,5,2022-03-21,USAN1034654-2,遥控器,邓园园,eBay-yaqinsale11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1602,1485,2022-05-09,112-3112726-2556257,USAN1011085-4,"PartP（排骨条卡扣）*12,PartQ（排骨条卡扣）*24",金晓强,赫曼,NaN,1485,2022-05-09,USAN1011085-4,"PartP（排骨条卡扣）*12,PartQ（排骨条卡扣）*24",金晓强,赫曼,NaN
1603,1487,2022-05-09,112-5375617-8366622,USAN1023811-5,床腿H，挂片K*2,李莹莹,信盒,NaN,1487,2022-05-09,USAN1023811-5,床腿H，挂片K*2,李莹莹,信盒,NaN
1604,1488,2022-05-09,114-7904736-4051433,USAN1026261-2,O塑料固定片*28,严鹏程,宫本,NaN,1488,2022-05-09,USAN1026261-2,O塑料固定片*28,严鹏程,宫本,NaN
1605,1489,2022-05-09,CS377684547,USAN1029204-1,轮子*6,屠乐天,Wayfair-信盒贸易,NaN,1489,2022-05-09,USAN1029204-1,轮子*6,屠乐天,Wayfair-信盒贸易,NaN


In [26]:
 loudeng.to_excel(excel_writer="D:/work/国内补寄/国内补寄校对/"+now+r"售后漏登记.xlsx", index=False,encoding='utf-8')

In [55]:
# import datetime
# AdDate = datetime.date( 2022, 1, 1)
# inventory_buji_big['订单时间'] = pd.to_datetime(inventory_buji_big['订单时间'])
# inventory_buji_big['补寄订单时间'] = pd.to_datetime(inventory_buji_big['补寄订单时间'])
# # inventory_1 = inventory_buji_big[inventory_buji_big['订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time())  ]
# # inventory_2 = inventory_buji_big[inventory_buji_big['补寄订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time())]
# # inventory_3 = pd.merge(inventory_1,inventory_2,how = 'outer', on = ['平台订单号','账号','平台SKU','公司SKU','数量','包裹号','订单类型','订单状态','补寄sku数量','补寄配件次数','补寄数量'])
inventory_3

,ID,日期,订单号,SKU,详情,寄件人,店铺,备注
0,1,2022-01-27,112-6466044-4167449,USAN1032096-11,床头柱B2,胡程程,赫曼,NaN
1,2,2022-01-25,114-4681848-2996252,USAN1026259-3,partJ,黄平,信盒,NaN
2,3,2022-01-11,CS378515836,USAN1029122-2,连接扣O2，O1,王荣荣,Wayfair-信盒贸易,NaN
3,4,2022-03-07,06-08329-92564,USAN1025836-1,螺丝partE.F,曹玲玲,eBay-zhirun2021,NaN
4,5,2022-03-21,08-08324-81027,USAN1034654-2,遥控器,邓园园,eBay-yaqinsale11,NaN
...,...,...,...,...,...,...,...,...
1478,1485,2022-05-09,112-3112726-2556257,USAN1011085-4,"PartP（排骨条卡扣）*12,PartQ（排骨条卡扣）*24",金晓强,赫曼,NaN
1479,1487,2022-05-09,112-5375617-8366622,USAN1023811-5,床腿H，挂片K*2,李莹莹,信盒,NaN
1480,1488,2022-05-09,114-7904736-4051433,USAN1026261-2,O塑料固定片*28,严鹏程,宫本,NaN
1481,1489,2022-05-09,CS377684547,USAN1029204-1,轮子*6,屠乐天,Wayfair-信盒贸易,NaN


In [56]:
inventory_list =inventory_3['订单号'].drop_duplicates()
inventory_list

0       112-6466044-4167449
1       114-4681848-2996252
2               CS378515836
3            06-08329-92564
4            08-08324-81027
               ...         
1478    112-3112726-2556257
1479    112-5375617-8366622
1480    114-7904736-4051433
1481            CS377684547
1482    113-3757248-1126622
Name: 订单号, Length: 1425, dtype: object

In [57]:
import time
#2022年售后漏登
# dengji_buji1 = pd.DataFrame(columns=['日期','寄件人','小组','店铺','订单号','地址','电话（亚马逊订单上的电话是虚拟号，打不通，一定要问到真实的电话号码）','sku','配件中文名称','追踪号','是否工厂支付运费','重量（KG）','RMB','USD','快递公司','国内补寄/国外补寄','校验','自发货系统SKU'])
# inventory_in_buji1 =  pd.DataFrame(columns=['日期','寄件人','小组','店铺','订单号','地址','电话（亚马逊订单上的电话是虚拟号，打不通，一定要问到真实的电话号码）','sku','配件中文名称','追踪号','是否工厂支付运费','重量（KG）','RMB','USD','快递公司','国内补寄/国外补寄','校验','自发货系统SKU'])
dengji_buji1 = pd.DataFrame(columns=['ID','日期','订单号','SKU','详情','寄件人','店铺'])
inventory_in_buji1 =  pd.DataFrame(columns=['ID','日期','订单号','SKU','详情','寄件人','店铺'])
for i in range(len(inventory_3)):
    
    if (str(inventory_3.iloc[i,2]) in origin_buji_big['订单号'].tolist()):
        
        if inventory_3.iloc[i,2] == '7863636525433':
            print('asdfdhghk')
        origin_output1 = pd.DataFrame(inventory_3[inventory_3['订单号']==inventory_3.iloc[i,2]])
  
        dengji_buji1=dengji_buji1.append(origin_output1)

    else:
        print(i)
        origin_output = pd.DataFrame(inventory_3[inventory_3['订单号']==inventory_3.iloc[i,2]])
        inventory_in_buji1=inventory_in_buji1.append(origin_output)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [58]:
# dengji_buji1 = pd.DataFrame(columns=['ID','日期','订单号','SKU','详情','寄件人','店铺'])

# if ('112-2617617-3593822' in origin_buji_big['订单号'].tolist()):
#     print(1)
#     origin_output1 = pd.DataFrame(inventory_3[inventory_3['订单号']=='112-2617617-3593822'])

#     dengji_buji1=dengji_buji1.append(origin_output1)

# else:
#     print(2)
#     origin_output = pd.DataFrame(inventory_3[inventory_3['订单号']=='112-2617617-3593822'])
#     inventory_in_buji1=inventory_in_buji1.append(origin_output)

In [59]:
inventory_in_buji1

,ID,日期,订单号,SKU,详情,寄件人,店铺,备注
0,1,2022-01-27 00:00:00,112-6466044-4167449,USAN1032096-11,床头柱B2,胡程程,赫曼,NaN
1,2,2022-01-25 00:00:00,114-4681848-2996252,USAN1026259-3,partJ,黄平,信盒,NaN
2,3,2022-01-11 00:00:00,CS378515836,USAN1029122-2,连接扣O2，O1,王荣荣,Wayfair-信盒贸易,NaN
3,4,2022-03-07 00:00:00,06-08329-92564,USAN1025836-1,螺丝partE.F,曹玲玲,eBay-zhirun2021,NaN
4,5,2022-03-21 00:00:00,08-08324-81027,USAN1034654-2,遥控器,邓园园,eBay-yaqinsale11,NaN
...,...,...,...,...,...,...,...,...
1478,1485,2022-05-09 00:00:00,112-3112726-2556257,USAN1011085-4,"PartP（排骨条卡扣）*12,PartQ（排骨条卡扣）*24",金晓强,赫曼,NaN
1479,1487,2022-05-09 00:00:00,112-5375617-8366622,USAN1023811-5,床腿H，挂片K*2,李莹莹,信盒,NaN
1480,1488,2022-05-09 00:00:00,114-7904736-4051433,USAN1026261-2,O塑料固定片*28,严鹏程,宫本,NaN
1481,1489,2022-05-09 00:00:00,CS377684547,USAN1029204-1,轮子*6,屠乐天,Wayfair-信盒贸易,NaN


In [60]:
now = time.strftime("%Y-%m-%d-%H_%M",time.localtime(time.time())) 

# dengji_buji = dengji_buji.drop_duplicates(keep = False)

dengji_buji1.to_excel(excel_writer="D:/work/国内补寄/国内补寄校对/"+now+r"补寄清单售后已登记.xlsx", index=False,encoding='utf-8')
# inventory_in_buji = inventory_in_buji.drop_duplicates(keep = 'first')
inventory_in_buji1.to_excel(excel_writer="D:/work/国内补寄/国内补寄校对/"+now+r"补寄清单售后漏登记.xlsx", index=False,encoding='utf-8')

In [193]:
inventory_in_buji1[inventory_in_buji1['订单号'] == '7863636525433']

,ID,日期,订单号,SKU,详情,寄件人,店铺


In [194]:
inventory_in_buji1

,ID,日期,订单号,SKU,详情,寄件人,店铺
1203,1206,2022-04-11,112-3560313-8696234,USAN1023893-2,partF床杆,何晓敏,赫曼
1417,1420,2022-05-01,112-3560313-8696234,USAN1023893-2,part F 床杆,何晓敏,赫曼
1235,1238,2022-04-14,114-7015379-7270631,USAN1032096-12,G,夏银平,赫曼
1276,1279,2022-04-18,112-2470183-4240222,USAN1023384-2,遥控器1，适配器1,孙紫馨,信盒
1313,1316,2022-04-20,5871050452741,USAN1026301-1,"缺少E*2,F*4,G*2",李永芳,Walmart-赫曼
1365,1368,2022-04-26,112-1400641-6362641,USAN1034691-2,U抽屉,何晓敏,赫曼
1383,1386,2022-04-27,114-7850372-1517814,USAN1023944-2,I*2，H*1,屠乐天,赫曼
1401,1404,2022-04-29,113-3767677-1458612,USAN1034691-5,partJ和partI2,何晓敏,赫曼
1420,1423,2022-05-01,113-3767677-1458612,USAN1034691-5,partJ和partI2,何晓敏,赫曼
1203,1206,2022-04-11,112-3560313-8696234,USAN1023893-2,partF床杆,何晓敏,赫曼
